In [2]:
# Add the path to the 'my_llm' module
import sys
sys.path.append('../src')

from my_llm.my_utils import load_all_jsonl_files
from my_llm.my_utils import load_normalized_jsonl
from my_llm.my_utils import load_txt_files_to_df
from my_llm.my_utils import split_train_test
from my_llm.my_utils import sample_and_print
from my_llm.my_utils import save_df_to_jsonl


from my_llm.gpt2_client import train_gpt2_model_from_df
from my_llm.my_utils import clean_dataframe
from my_llm.gpt_client import pre_trained_model
from my_llm.llama_client import train_llama_model_from_df
from my_llm.deepdistil_client import train_deep_model_from_df
#from my_llm.flan_client import train_flan_model_from_df

from datasets import Dataset
from sklearn.model_selection import train_test_split
import re
import glob
import os
import pandas as pd
from transformers import pipeline, Trainer



2025-04-07 19:59:00.332098: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Load configuration in a Python file (e.g., main.py or a notebook cell)
import yaml

with open("/Users/blairjdaniel/AI-Assistant-Springs/config/model_config.yaml", "r") as file:
    config = yaml.safe_load(file)

print(config)

{'split': {'test_size': 0.2, 'validation_size': 0.1, 'random_state': 42}, 'pipeline': {'task': 'text-generation', 'model_name': 'EleutherAI/gpt-neo-1.3B'}, 'model': {'name': 'EleutherAI/gpt-neo-1.3B', 'training': {'epochs': 3, 'batch_size': 32, 'learning_rate': '2e-5', 'max_seq_length': 128}}, 'tokenizer': {'name': 'EleutherAI/gpt-neo-1.3B', 'do_lower_case': True, 'truncation': True, 'padding': 'max_length'}, 'huggingface': {'cache_dir': '~/.cache/huggingface', 'use_auth_token': False}, 'logging': {'level': 'INFO', 'log_file': 'logs/training.log'}}


In [4]:
def load_model_config(yaml_path: str) -> dict:
    with open(yaml_path, "r") as file:
        return yaml.safe_load(file)

In [5]:
# # Load the config
# with open("/Users/blairjdaniel/AI-Assistant-Springs/config/model_config.yaml", "r") as f:
#     model_config = yaml.safe_load(f)

# pipe = pipeline(
#     model_config["pipeline"]["task"],
#     model=model_config["pipeline"]["model_name"]
#)

In [6]:
# file_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/email_data/data_cleaned/data_email.json"
# normalized_data = load_normalized_jsonl(file_path)
# print("Normalized DataFrame shape:", normalized_data.shape)

In [ ]:
file_path = ""
data = load_normalized_jsonl(file_path)
print("Normalized DataFrame shape:", data.shape)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split



# Split the data into train and test sets
train_data, test_data = train_test_split(
    data,
    test_size=config['split']['test_size'],  # Use test size from config
    random_state=config['split']['random_state']  # Use random state from config
)

# Further split the training data into train and validation sets
train_data, val_data = train_test_split(
    train_data,
    test_size=config['split']['validation_size'],  # Use validation size from config
    random_state=config['split']['random_state']
)

import os

# Ensure the directory exists
output_dir_path = '/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab'
os.makedirs(output_dir_path, exist_ok=True)

import pandas as pd

# Convert lists to pandas DataFrames
train_data_df = pd.DataFrame(train_data)
val_data_df = pd.DataFrame(val_data)
test_data_df = pd.DataFrame(test_data)

# Save the datasets to JSON files
train_data_df.to_json(os.path.join(output_dir_path, 'attitude_train.json'), orient='records', lines=True)
val_data_df.to_json(os.path.join(output_dir_path, 'attitude_val.json'), orient='records', lines=True)
test_data_df.to_json(os.path.join(output_dir_path, 'attitude_test.json'), orient='records', lines=True)


In [ ]:
import pandas as pd
import torch
# Load model directly
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased-distilled-squad")



# Prepare dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.encodings["input_ids"][idx]),
            "attention_mask": torch.tensor(self.encodings["attention_mask"][idx]),
            "labels": torch.tensor(self.labels["input_ids"][idx]),
        }

train_dataset = CustomDataset(train_encodings, labels)

model = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-cased-distilled-squad")

# Define training arguments
training_args = TrainingArguments(
    output_dir="/Users/blairjdaniel/AI-Assistant-Springs/models/flan-5",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
)

# Train model
# Tokenize validation data
val_encodings = tokenizer(list(val_data["prompt"]), truncation=True, padding=True, max_length=128)
val_labels = tokenizer(list(val_data["completion"]), truncation=True, padding=True, max_length=128)

# Prepare validation dataset
val_dataset = CustomDataset(val_encodings, val_labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # Add validation dataset here
)
trainer.train()

In [23]:
# # Define output directory
# output_dir = "/Users/blairjdaniel/AI-Assistant-Springs/models/gpt2_finetuned_four"

# # Train the model
# train_gpt2_model_from_df(train_data, val_data, output_dir, num_train_epochs=5)

In [24]:
# # Define output directory
# output_dir = "/Users/blairjdaniel/AI-Assistant-Springs/models/llama"

# # Train the model
# train_llama_model_from_df(train_data, val_data, output_dir, num_train_epochs=3)

In [25]:
# # Define output directory
# output_dir = "/Users/blairjdaniel/AI-Assistant-Springs/models/deepseek_distil"

# # Train the model
# train_deep_model_from_df(train_data, val_data, output_dir, num_train_epochs=3)

In [26]:
# # Use a pipeline as a high-level helper
# from transformers import pipeline

# messages = [
#     {"role": "user", "content": "Who are you?"},
# ]
# pipe = pipeline("text-generation", model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
# pipe(messages)

In [27]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b")
# model = AutoModelForCausalLM.from_pretrained("google/gemma-7b")



In [28]:
# prompt = "Generate a response to: 'What is the capital of France?'"
# input_ids = tokenizer(prompt, return_tensors="pt").input_ids
# outputs = model.generate(input_ids, max_length=100)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [31]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("/Users/blairjdaniel/AI-Assistant-Springs/models/gemma-7")
tokenizer = AutoTokenizer.from_pretrained("/Users/blairjdaniel/AI-Assistant-Springs/models/gemma-7")

# prompt = "Generate a response to: 'What is the capital of France?'"
# tokenizer.pad_token = tokenizer.eos_token
# inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=1024)
# outputs = model.generate(
#     inputs["input_ids"],
#     attention_mask=inputs["attention_mask"],
#     max_length=1024,
#     min_length=50,
#     temperature=0.5,
#     do_sample=True,
#     pad_token_id=tokenizer.pad_token_id,
#     repetition_penalty = 1.2
# )
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
prompt = "Generate a response to: 'What is the capital of France?'"
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=1024)
outputs = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=1024,
    min_length=50,
    temperature=0.5,
    do_sample=True,
    pad_token_id=tokenizer.pad_token_id,
    repetition_penalty = 1.2
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
# # Save locally
# local_dir = "/Users/blairjdaniel/AI-Assistant-Springs/models/gemma-7"

# model.save_pretrained(local_dir)
# tokenizer.save_pretrained(local_dir)

# print("Model and processor saved locally at:", local_dir)

In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# model_name = "EleutherAI/gpt-neo-1.3B"  # or "EleutherAI/gpt-neo-1.3B"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

# prompt = "Generate a response to: 'What is the capital of France?'"
# input_ids = tokenizer(prompt, return_tensors="pt").input_ids
# outputs = model.generate(input_ids, max_length=100)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# # Save locally
# local_dir = "/Users/blairjdaniel/AI-Assistant-Springs/models/neo"

# model.save_pretrained(local_dir)
# tokenizer.save_pretrained(local_dir)

# print("Model and processor saved locally at:", local_dir)

In [ ]:
# df = pd.read_csv('/Users/blairjdaniel/AI-Assistant-Springs/data/google_colab/finals_all.csv')

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
from evaluate import load
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
import numpy as np

# 3. Convert your DataFrame into a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Optionally, split your dataset into train and test sets
dataset = dataset.train_test_split(test_size=0.2, seed=42)
datasets = DatasetDict({
    "train": dataset["train"],
    "test": dataset["test"]
})

# 4. Load DistilBERT tokenizer and preprocess the texts
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

def tokenize_function(example):
    return tokenizer(example["subject"], padding="max_length", truncation=True, max_length=64)

tokenized_datasets = datasets.map(tokenize_function, batched=True)

# Set the format for PyTorch
tokenized_datasets = tokenized_datasets.remove_columns(["subject", "label_text"])
tokenized_datasets.set_format("torch")

# 5. Load the DistilBERT model for sequence classification with 4 labels
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=4)

accuracy_metric = load("accuracy")
accuracy_metric = load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

# 7. Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# 8. Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

# 9. Train the model
trainer.train()

# Optionally: evaluate on test set
evaluation_result = trainer.evaluate()
print("Evaluation results:", evaluation_result)

# Save the model if needed:
model.save_pretrained("/Users/blairjdaniel/AI-Assistant-Springs/models/distilbert-email-classifier")
tokenizer.save_pretrained("/Users/blairjdaniel/AI-Assistant-Springs/models/distilbert-email-classifier")


In [ ]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2-large")
# model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2-large")

# # Save locally
# local_dir = "/Users/blairjdaniel/AI-Assistant-Springs/models/gpt"

# model.save_pretrained(local_dir)
# tokenizer.save_pretrained(local_dir)

# print("Model and processor saved locally at:", local_dir)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model and processor saved locally at: /Users/blairjdaniel/AI-Assistant-Springs/models/gpt
